```{contents}
```
## RunnableParallel

### What Is RunnableParallel

**RunnableParallel** is a LangChain runnable that allows you to **execute multiple runnables concurrently on the same input** and collect all their outputs into a single structured result.

It is a **fan-out / fan-in primitive** in LCEL (LangChain Expression Language).

Provided by LangChain.

```
Single Input
   ├── Runnable A
   ├── Runnable B
   ├── Runnable C
   ↓
Combined Output (dict)
```

---

### Why RunnableParallel Is Needed

In real pipelines, you often want to:

* Run retrieval and preprocessing together
* Extract multiple features from the same input
* Call multiple tools/models in parallel
* Reduce latency

RunnableParallel solves:

* Sequential bottlenecks
* Manual threading/async code
* Complex wiring logic

---

### Conceptual Flow

![Image](https://dz2cdn1.dzone.com/storage/temp/18047285-screenshot-2024-11-18-at-121208pm.png)

![Image](https://www.pinecone.io/_next/image/?q=75\&url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2F63f8a8482c9ec06a8d7d1041514f87c06dd108a9-3442x942.png\&w=3840)

![Image](https://www.gocd.org/assets/images/blog/build-propagation/fan-in-fan-out-774eac6e.jpeg)

```
Input Query
   ├─► Retriever
   ├─► Metadata Extractor
   └─► Normalizer
        ↓
   { context, metadata, normalized_query }
```

---

## Demonstrations

---

### Basic RunnableParallel (Minimal)

```python
from langchain_core.runnables import RunnableParallel, RunnableLambda

parallel = RunnableParallel(
    upper=RunnableLambda(lambda x: x.upper()),
    length=RunnableLambda(lambda x: len(x))
)

parallel.invoke("LangChain")
```

**Output**

```python
{
  "upper": "LANGCHAIN",
  "length": 9
}
```

All runnables:

* Receive the **same input**
* Run **in parallel**
* Return a dictionary

---

### RunnableParallel Using LCEL Dictionary Syntax

RunnableParallel is commonly written using a **dict literal**:

```python
parallel = {
    "original": RunnableLambda(lambda x: x),
    "lower": RunnableLambda(lambda x: x.lower()),
    "words": RunnableLambda(lambda x: x.split())
}

parallel.invoke("Runnable Parallel Demo")
```

**Output**

```python
{
  "original": "Runnable Parallel Demo",
  "lower": "runnable parallel demo",
  "words": ["Runnable", "Parallel", "Demo"]
}
```

---

### RunnableParallel in a RAG Pipeline (Very Common)

### Problem

You need:

* Original question
* Retrieved context
* Possibly other derived data

---

### Solution

```python
from langchain_core.runnables import RunnablePassthrough

chain = (
    {
        "question": RunnablePassthrough(),
        "context": retriever
    }
    | prompt
    | llm
)
```

This `{ ... }` block is a **RunnableParallel**.

What happens:

* Question → passed through unchanged
* Question → sent to retriever
* Both results are merged

---

### RunnableParallel with Multiple Models

```python
from langchain_openai import ChatOpenAI

llm_fast = ChatOpenAI(model="gpt-3.5-turbo")
llm_accurate = ChatOpenAI(model="gpt-4")

parallel = RunnableParallel(
    fast=llm_fast,
    accurate=llm_accurate
)

parallel.invoke("Explain RAG briefly")
```

**Use case**

* Compare responses
* Voting / re-ranking
* Cost vs accuracy trade-off

---

### RunnableParallel + RunnableLambda (Feature Extraction)

```python
parallel = RunnableParallel(
    char_count=RunnableLambda(lambda x: len(x)),
    word_count=RunnableLambda(lambda x: len(x.split())),
    has_question_mark=RunnableLambda(lambda x: "?" in x)
)

parallel.invoke("What is RunnableParallel?")
```

**Output**

```python
{
  "char_count": 25,
  "word_count": 3,
  "has_question_mark": True
}
```

---

### Execution Semantics

* All branches receive **identical input**
* Execution is **concurrent** (async where supported)
* Failure in one branch fails the whole runnable
* Output is always a **dictionary**

---

### RunnableParallel vs Sequential Chains

| Aspect    | RunnableParallel | Sequential     |
| --------- | ---------------- | -------------- |
| Execution | Concurrent       | One-by-one     |
| Latency   | Low              | Higher         |
| Output    | Dict             | Single value   |
| Use case  | Fan-out          | Transformation |

---

### When to Use RunnableParallel

Use it when:

* Same input feeds multiple steps
* You need speed
* You want structured outputs
* Building RAG / agents / evaluators

Avoid it when:

* Steps depend on each other
* Order matters
* Output of one step feeds the next

---

### Common Real-World Use Cases

* RAG: question + context
* Agent state construction
* Guardrails + normalization
* Multi-model evaluation
* Feature extraction pipelines

---

### Common Mistakes

* Expecting branches to see each other’s output
* Using it when sequential dependency exists
* Forgetting outputs are keyed

---

### Mental Model

RunnableParallel is the **“map” stage** of an LLM pipeline.

```
One input → many computations → one structured state
```

---

### Key Takeaways

* RunnableParallel enables **fan-out / fan-in**
* Core primitive of LCEL
* Written using `{}` syntax most of the time
* Essential for RAG and agent pipelines
